In [2]:
import requests
from datetime import date
import datetime
from datetime import datetime  
from datetime import timedelta  
import dateutil.parser
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
%matplotlib inline
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [3]:
## Time horizon D+x for making regression predictions
Prediction_Horizon = 14
## Starting point for minimal number of confirmed cases
Min_Confirmed = 10

## Number of days used for making the linear prediction
Days = 7

In [4]:
url = 'https://data.humdata.org/hxlproxy/data/download/time_series_covid19_confirmed_global_narrow.csv?dest=data_edit&filter01=explode&explode-header-att01=date&explode-value-att01=value&filter02=rename&rename-oldtag02=%23affected%2Bdate&rename-newtag02=%23date&rename-header02=Date&filter03=rename&rename-oldtag03=%23affected%2Bvalue&rename-newtag03=%23affected%2Binfected%2Bvalue%2Bnum&rename-header03=Value&filter04=clean&clean-date-tags04=%23date&filter05=sort&sort-tags05=%23date&sort-reverse05=on&filter06=sort&sort-tags06=%23country%2Bname%2C%23adm1%2Bname&tagger-match-all=on&tagger-default-tag=%23affected%2Blabel&tagger-01-header=province%2Fstate&tagger-01-tag=%23adm1%2Bname&tagger-02-header=country%2Fregion&tagger-02-tag=%23country%2Bname&tagger-03-header=lat&tagger-03-tag=%23geo%2Blat&tagger-04-header=long&tagger-04-tag=%23geo%2Blon&header-row=1&url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv'
myfile = requests.get(url)
open(r'C:\Users\JMSch\Documents\MEGA\05_Paris\ESCP Business School\MSc Big Data & Business Analytics\COVID19\COVID19\Data\time_series_covid19_confirmed_global_narrow(all).csv', 'wb').write(myfile.content)

827391

In [5]:
data = pd.read_csv(r'C:\Users\JMSch\Documents\MEGA\05_Paris\ESCP Business School\MSc Big Data & Business Analytics\COVID19\COVID19\Data\time_series_covid19_confirmed_global_narrow(all).csv', sep = ',')
data = data[1:]

## Choosing the columns for later use
data = data.filter(['Country/Region', 'Date', 'Value'])
## Renaming the columns we are going to use
data.columns = ['Country', 'Date', 'Value']
## Transforming the value to int
data['Value'] = data['Value'].astype(int)
data['Date'] = data['Date'].astype('datetime64[D]')
## Summing values for 'Value' for which there are multiple records for one coutry on one day
data['Total'] = data.groupby(['Country', 'Date'])['Value'].transform('sum')
## Delete the duplicate records
data = data.drop_duplicates(subset=['Country', 'Date'])
## Sort the value from earliest to latest
data = data.sort_values(['Date'], ascending = True)
data.columns = ['Country', 'Date', 'Value', 'Confirmed']

In [6]:
## Remove the dates of which no infections were reported
data = data[data['Confirmed'] > Min_Confirmed]

## Transform the date from Excel numbers to Python dates
data['Date'] = data['Date'].astype('datetime64[D]') 
data['Date'] = data['Date'].dt.strftime('%d/%m/%Y')

In [7]:
data['Time'] = 1
data = data.sort_values(by=['Country', 'Confirmed'])
data.index = range(len(data))

for index, row in data.iloc[1:].iterrows():
    if data.loc[index, 'Country'] == data.loc[(int(index) - 1), 'Country']: 
        data_new = data[data.Country.eq(data.loc[index, 'Country'])]
        HighestForCountry = data_new['Time'].max()
        data.loc[index, 'Time'] = HighestForCountry + 1
    else:
        data.loc[index, 'Time'] = 1

In [8]:
Countries = data['Country'].tolist()
Countries = list(dict.fromkeys(Countries))

In [9]:
Full_Data = pd.DataFrame()

## For each country, find the 'Time' reference since patient 1, and the corrsponding date
for Country in Countries:
    data_sub = data[data['Country'].eq(Country)]
    Current_Horizon = int(max(data_sub['Time']))
    Days = Current_Horizon
    FindDate = data_sub['Date'].iloc[-1]
    FindDate = datetime.strptime(FindDate, "%d/%m/%Y").strftime("%Y-%m-%d")
    MaxDate = dateutil.parser.parse(FindDate).date()

## For each country, add the number of days to the prediction horizon of the dataset based on the 'Time' reference and date
    for i in range(Prediction_Horizon):
        New_Time = int(max(data_sub['Time'])) + 1
        Days_Delay = (New_Time - Current_Horizon)
        New_Date = MaxDate + timedelta(days = Days_Delay)
        New_Date = New_Date.strftime("%d/%m/%Y")
        data_sub = data_sub.append({'Country': Country,'Date': New_Date, 'Time': New_Time},ignore_index=True)
    Full_Data = pd.concat([Full_Data,data_sub]) 


In [10]:
def f_sigmoid(x, a, b, c):
    # a = sigmoid midpoint
    # b = curve steepness (logistic growth)
    # c = max value
        return (c / (1 + np.exp(-b*(x-a))))

In [11]:
## building the model per country
Pred0_Data = pd.DataFrame()
for Country in Countries:
    data_cntry = Full_Data[Full_Data.Country.eq(Country)]
    if data_cntry.count(axis='columns').count() > 0:
        try:
            
            data_cntry = data_cntry[['Country', 'Time','Confirmed','Date']]
  
            Model_Data = data_cntry.dropna()
            x = Model_Data['Time'].values
            y = Model_Data['Confirmed'].values

            popt_sig, pcov_sig = curve_fit(f_sigmoid, x, y, method='dogbox', bounds=([20., 0.001, y.mean()],[60., 2.5, 10*y.max()]), maxfev=5000)

            Pred_Length = data_cntry['Confirmed'].values

            x_m = np.arange(len(Pred_Length))
            y_m = f_sigmoid(x_m, *popt_sig)

            data_cntry['Logistic Prediction']=pd.Series(y_m)

            Pred0_Data = pd.concat([Pred0_Data,data_cntry]) 

        except:
           continue

    else:
            continue


In [12]:
def Polynomial_Regression(t):
    return pol_reg.predict(poly_reg.fit_transform([[t]]))

In [13]:
## building the model per country
Pred1_Data = pd.DataFrame()

for Country in Countries:
    data_cntry = Pred0_Data[Pred0_Data.Country.eq(Country)]
    if data_cntry.count(axis='columns').count() > 8:
        try:
            Model_Data = data_cntry.dropna()
            Model_Data = Model_Data[['Time','Confirmed']]

            X = Model_Data[['Time']]
            y = Model_Data[['Confirmed']]

            X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=0)

            poly_reg = PolynomialFeatures(degree=4)
            X_poly = poly_reg.fit_transform(X)
            pol_reg = LinearRegression()
            pol_reg.fit(X_poly, y)

            ## This code transforms the regression coefficients into floats to use in our prediction function

            data_cntry.insert(5,'Polynomial Prediction',0)
            data_cntry['Polynomial Prediction'] = data_cntry['Time'].apply(Polynomial_Regression)
            
            ## remove first set of square brackets from prediciton output
            data_cntry['Polynomial Prediction'] = data_cntry['Polynomial Prediction'].str[0]
            ## remove second set of square brackets from prediciton output
            data_cntry['Polynomial Prediction'] = data_cntry['Polynomial Prediction'].str[0]
            ## 
            data_cntry['Polynomial Prediction'] =  data_cntry['Polynomial Prediction'].astype(float)

            Pred1_Data = pd.concat([Pred1_Data,data_cntry]) 
        except:
            continue

    else:
            continue


In [14]:
def linear_predictions(t):
    return np.exp(Constant_Coefficient) * np.exp(Time_Coeffificient) ** t

In [21]:
## building the model per country
Pred2_Data = pd.DataFrame()

Pred1_Data['logConfirmed'] = np.log(Pred1_Data.Confirmed)

for Country in Countries:
    data_cntry = Pred1_Data[Pred1_Data.Country.eq(Country)]
    if data_cntry.count(axis='columns').count() > 8:
        try:
            Model_Data = data_cntry.dropna()
            Last_Date = int(max(Model_Data['Time']))
            Days_Delay = (Last_Date - Days)
            Model_Data = Model_Data.iloc[Days_Delay:]

            X = Model_Data.Time
            X = sm.add_constant(X)
            y = Model_Data.logConfirmed

            mod = sm.OLS(y, X)
            res = mod.fit()
            results_summary = res.summary()
            
            ## This code transforms the regression coefficients into floats to use in our prediction function
            results_as_html = results_summary.tables[1].as_html()
            New_DF = pd.read_html(results_as_html, header=0, index_col=0)[0]

            Constant_Coefficient = New_DF.at['const','coef']
            Time_Coeffificient = New_DF.at['Time','coef']

            data_cntry['Linear Prediction'] = data_cntry.Time.apply(linear_predictions)
    
            Pred2_Data = pd.concat([Pred2_Data,data_cntry]) 
        except:
            continue
    else:
            continue


C:\Python38\lib\site-packages\statsmodels\stats\stattools.py:70: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Python38\lib\site-packages\statsmodels\stats\stattools.py:70: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Python38\lib\site-packages\statsmodels\stats\stattools.py:70: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Python38\lib\site-packages\statsmodels\stats\stattools.py:70: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Python38\lib\site-packages\statsmodels\stats\stattools.py:70: ValueWarning: o

In [22]:
Pred2_Data

,Country,Time,Confirmed,Date,Logistic Prediction,Polynomial Prediction,logConfirmed,Linear Prediction
0,Afghanistan,1,11.0,14/03/2020,8.420491e+00,14.985846,2.397895,16.003381
1,Afghanistan,2,16.0,15/03/2020,1.010630e+01,15.001318,2.772589,18.496842
2,Afghanistan,3,21.0,16/03/2020,1.212575e+01,15.869337,3.044522,21.378805
3,Afghanistan,4,22.0,17/03/2020,1.454314e+01,17.613556,3.091042,24.709802
4,Afghanistan,5,22.0,18/03/2020,1.743447e+01,20.276083,3.091042,28.559796
...,...,...,...,...,...,...,...,...
19,Zambia,20,NaN,13/04/2020,8.250826e-10,391.351399,NaN,160.581243
20,Zambia,21,NaN,14/04/2020,2.881261e-09,507.136364,NaN,181.453605
21,Zambia,22,NaN,15/04/2020,1.006162e-08,648.961538,NaN,205.038958
22,Zambia,23,NaN,16/04/2020,3.513607e-08,820.389860,NaN,231.689937


In [23]:
Pred2_Data = Pred2_Data.dropna()
Countries = Pred2_Data['Country'].tolist()
Countries = list(dict.fromkeys(Countries))
Pred2_Data

,Country,Time,Confirmed,Date,Logistic Prediction,Polynomial Prediction,logConfirmed,Linear Prediction
0,Afghanistan,1,11.0,14/03/2020,8.420491e+00,14.985846,2.397895,16.003381
1,Afghanistan,2,16.0,15/03/2020,1.010630e+01,15.001318,2.772589,18.496842
2,Afghanistan,3,21.0,16/03/2020,1.212575e+01,15.869337,3.044522,21.378805
3,Afghanistan,4,22.0,17/03/2020,1.454314e+01,17.613556,3.091042,24.709802
4,Afghanistan,5,22.0,18/03/2020,1.743447e+01,20.276083,3.091042,28.559796
...,...,...,...,...,...,...,...,...
5,Zambia,6,35.0,30/03/2020,2.057330e-17,33.604895,3.555348,29.020428
6,Zambia,7,35.0,31/03/2020,7.184378e-17,35.578671,3.555348,32.792506
7,Zambia,8,36.0,01/04/2020,2.508848e-16,36.840909,3.583519,37.054878
8,Zambia,9,39.0,02/04/2020,8.761121e-16,37.870629,3.663562,41.871274


In [24]:
def adj_r_squared(pred,real):
    correlation_matrix = np.corrcoef(pred, real)
    correlation_xy = correlation_matrix[0,1]
    r_squared = correlation_xy**2

    return 1-(1-r_squared)*(n-1)/(n-p-1)

In [55]:
## For each country, find the 'Time' reference since patient 1, and the corrsponding date
Output = pd.DataFrame() #Temporary empty dataframe
Cols = ['Country', 'Logistic R2', 'Linear R2', 'Polynomial R2']
Output = pd.DataFrame(columns = Cols)

for Country in Countries:
    data_cntry = Pred2_Data[Pred2_Data.Country.eq(Country)]

    Logŷ = data_cntry['Logistic Prediction'].values
    Linŷ = data_cntry['Linear Prediction'].values
    Polŷ = data_cntry['Polynomial Prediction'].values
    y = data_cntry['Confirmed'].values

    n = data_cntry.count(axis='columns').count()
    p = 1

    LogAdjR2 = adj_r_squared(Logŷ, y)
    LinAdjR2 = adj_r_squared(Linŷ, y)
    PolAdjR2 = adj_r_squared(Polŷ, y)

    Output = Output.append({'Country': Country, 'Logistic R2': LogAdjR2, 'Linear R2': LinAdjR2, 'Polynomial R2': PolAdjR2,}, ignore_index=True)

print(Output)

Country  Logistic R2  Linear R2  Polynomial R2
0           Afghanistan     0.983463   0.982623       0.983699
1               Albania     0.992650   0.988465       0.994508
2               Algeria     0.996653   0.996609       0.996420
3               Andorra     0.205897   0.957664       0.997101
4             Argentina     0.993357   0.984258       0.995822
..                  ...          ...        ...            ...
137          Uzbekistan     0.351400   0.976855       0.992996
138           Venezuela     0.166634   0.949988       0.989420
139             Vietnam     0.991450   0.970174       0.998012
140  West Bank and Gaza     0.978833   0.980709       0.987433
141              Zambia     0.151027   0.801060       0.987194

[142 rows x 4 columns]


In [56]:
now = datetime.now() # current date and time
Now = now.strftime("%Y%m%d_%Hh%M")

Pred2_Data.to_csv(f'C:\\Users\\JMSch\\Documents\\MEGA\\05_Paris\\ESCP Business School\\MSc Big Data & Business Analytics\\COVID19\\COVID19\\Data\\Extracts\\ALL_Prediction_Output_Global_{Now}.csv', index = False)

Output.to_csv(f'C:\\Users\\JMSch\\Documents\\MEGA\\05_Paris\\ESCP Business School\\MSc Big Data & Business Analytics\\COVID19\\COVID19\\Data\\Extracts\\Predictions_Adjusted_R-Squared_{Now}.csv', index = False)